In [ ]:
print("""This notebook is used to create the data files that are loaded in notebook 'Figure_8.ipynb'. As running the 
notebooks took a long time, we split the original notebook into 10: superspotter_fig_8_set_0, superspotter_fig_8_set_1 etc.
We only provide the set_0 version here for cleanness. If you want to recreate the other versions yourself, do as follows:

- Duplicate this notebook
- Change the copied notebook's name to 'superspotter_fig_8_set_{x}'
- In the new notebook, change the line 'input periods = list(np.logspace(np.log10(3), np.log10(40), 100))[0:10]'
into: 'input periods = list(np.logspace(np.log10(3), np.log10(40), 100))[{x}*10:{x+1}*10]'
- In the new notebook, change the line 'detections_file_name = 'detections_fig_8_set_0.csv' to
'detections_file_name = 'detections_fig_8_set_{x}.csv'

The notebook is now ready to run. Note that this may take a couple of days.""")

In [ ]:
#SUPERcomputer Searching PeriodOgrams for Transits To EmeRge (SUPER-SPOTTER)
import numpy as np
import os
import matplotlib.pyplot as plt
import batman
from tqdm import tqdm
from astropy import units as u
from astropy.io import ascii
from transitleastsquares import transitleastsquares

In [ ]:
#Get the data directory
cwd = os.getcwd()
data_dir = cwd.replace('Figure_8', 'Data\\')
data_file_name = '\Final_Combined_Data.csv' #The name of the input data file

input_periods = list(np.logspace(np.log10(3), np.log10(40), 100))[0:10] #The periodgrid
input_radii = list(np.linspace(0.5, 2, 11))                     #The radii grid
n_checks = 10                                                     #Amount of times to check each pixel

detections_file_name = 'detections_fig_8_set_0.csv'               #File name to save the number of detections file_name


#Define constants
r_star = 0.96*u.astrophys.R_sun.to(u.m)
m_star = 0.95*u.astrophys.M_sun.to(u.kg)
day_in_sec = u.day.to(u.second)
G = 6.6743*10**-11
r_jupiter = u.astrophys.jupiterRad.to(u.m)

#Reading in the data
data = ascii.read(data_dir + data_file_name)

mask = (data['col3'] < 0.05) #Removing measurements with high uncertainty
data = data[mask]

#Defining the times over which to produce fake data, the errors on the data and the transit parameters
times = data['col1']
#transit_params input: planet radius (jupiter radii), t_inferior_conjunction, orbital period (days), stellar mass (kg), stellar radius (m), inclination (degrees), eccentricity, longitude of periastron (degrees)
transit_params = [2, 5, 10, m_star, r_star, 90, 0, 90] 
e_flux = data['col3']

In [ ]:
def transit_time(period):
    a = float(((((period*day_in_sec)**2)*G*m_star)/(4*np.pi**2))**(1./3.))/r_star 
    return period/(np.pi*a)

#Generating noise and fake data
def noise_generator(times, std):
    return np.random.randn(times.size)*std #Generating Gaussian noise with a sigma of choice
       
def fake_data_generator(times, noise):
    data = np.ones_like(times) 
    data += noise
    return data

#Generating a fake transit
def transit_generator(times, r_planet, t0, per, m_star, r_star, inc, ecc, w):
    params = batman.TransitParams()              #object to store transit parameters
    params.t0 = float(t0)                        #time of inferior conjunction
    params.per = float(per)                      #orbital period
    params.rp = float(r_planet*r_jupiter)/r_star #planet radius (in units of stellar radii)
    params.a = float(((((per*day_in_sec)**2)*G*m_star)/(4*np.pi**2))**(1./3.))/r_star   #semi-major axis (in units of stellar radii)
    params.inc = float(inc)                      #orbital inclination (in degrees)
    params.ecc = float(ecc)                      #eccentricity
    params.w = float(w)                          #longitude of periastron (in degrees)
    params.limb_dark = "nonlinear"               #limb darkening model "nonlinear"
    params.u = [1.2, -0.47, -0.22, 0.24]         #limb darkening coefficients [u1, u2, u3, u4] for T4500 log4.0, standard = [0.5, 0.1, 0.1, -0.1] 

    t = times #times at which to calculate light curve 
    m = batman.TransitModel(params, t)    #initializes model
    return m.light_curve(params)

def tls(times, transit_flux, yerr, min_p, max_p, plot, stats): #tls 
    model = transitleastsquares(times, transit_flux, yerr)
    #Stellar parameters from Mentel 2018
    results = model.power(transit_depth_min = 0.001, use_threads = 96, R_star = 0.96, R_star_min = 0.81, R_star_max = 1.11, M_star = 0.95, M_star_min = 0.85, M_star_max = 1.05, period_min = min_p, period_max = max_p, oversampling_factor = 3, show_progress_bar = False)
    
    #Picking out the three most likely periods, excluding possible insignificantly different values
    picker = 1
    top_4_SDE = [0]
    while len(top_4_SDE) < 4:
        if len(top_4_SDE) == 3:
            if abs((results.periods[list(results.power).index(sorted(list(results.power))[-picker])]-results.periods[list(results.power).index(top_4_SDE[-1])])/(results.periods[list(results.power).index(top_4_SDE[-1])])) < 0.05 or abs((results.periods[list(results.power).index(sorted(list(results.power))[-picker])]-results.periods[list(results.power).index(top_4_SDE[-2])])/(results.periods[list(results.power).index(top_4_SDE[-2])])) < 0.05:
                pass
            else:
                top_4_SDE.append(sorted(list(results.power))[-picker])
        elif len(top_4_SDE) == 1:
            top_4_SDE.append(sorted(list(results.power))[-1])
        else:
            if abs((results.periods[list(results.power).index(sorted(list(results.power))[-picker])]-results.periods[list(results.power).index(top_4_SDE[-1])])/(results.periods[list(results.power).index(top_4_SDE[-1])])) < 0.05:
                pass
            else:
                top_4_SDE.append(sorted(list(results.power))[-picker])
        picker += 1
        
    top_3_SDE = top_4_SDE[1:]
    top_3_periods = [results.periods[list(results.power).index(top_3_SDE[0])], results.periods[list(results.power).index(top_3_SDE[1])], results.periods[list(results.power).index(top_3_SDE[2])]]

    
    return top_3_periods, top_3_SDE, results

def retrieved_condition(retrieved_p, inserted_p, sde): #A function that determines whether a fake transit was retrieved
    print("Retrieved periods: ", retrieved_p)
    print("SDE: ", sde)
    for i in range(3):
        if sde[i] > 6 and abs(retrieved_p[i]-inserted_p)/(inserted_p) < 0.05: 
            return True, i
    return False, -1

def iterator(periods, radii, n_checks, stats): #Function iterating over different periods and radii
        detections = np.zeros((n_checks, len(radii), len(periods)))  #Array to store scores
        for i in tqdm(periods):
            for j in tqdm(radii):
                retrieved = 0 #Counter for succesful iterations
                transit_params[0], transit_params[2] = j, i #Setting transit parameters to a certain period, radius
                for k in tqdm(range(n_checks)): #testing parameter set 'n_checks' amount of times
                    phase = np.random.uniform(0, i) #setting a random phase diff
                    transit_params[1] = phase
                    
                    flux = np.copy(data['col2']) #Creating fake transit flux
                    results = main(times, flux, e_flux, 3, i*2.1, False, stats, True, transit_params) #Looking for transit
                    detected, number = retrieved_condition(results[0], i, results[1])
                    if detected:
                        detections[k, radii.index(j), periods.index(i)] = 1
                    else:
                        detections[k, radii.index(j), periods.index(i)] = 0
                    
                    
                np.savetxt(data_dir + detections_file_name, np.sum(detections, axis=0), delimiter=',')
            
        return detections 

def start(periods, radii, n_checks, stats=False):
    return iterator(periods, radii, n_checks, stats)

def main(times, transit_flux, yerr, min_p, max_p, plot, stats, add_transit, transit_params):
    if add_transit:
        transit_flux += (transit_generator(times, *transit_params)-1)
    results = tls(times, transit_flux, yerr, min_p, max_p, plot, stats)
    return results

In [ ]:
heat_map_data = start(input_periods, input_radii, n_checks)